In [ ]:
!pip install swig
#!pip install Box2D-kengz
!pip install gym[box2d] pyglet==1.3.2
!pip install -U colabgymrender
!pip install stable-baselines3[extra]
!pip install renderlab
!pip install pyglet==1.3.2
!pip install pyvirtualdisplay
!apt-get install -y xvfb ffmpeg  # Removed python-opengl
!pip install PyOpenGL PyOpenGL_accelerate  # Install OpenGL via pip

!apt-get install -y xvfb x11-utils
!pip install pyvirtualdisplay gymnasium==0.28.1

# Import necessary libraries

from pyvirtualdisplay import Display
from IPython.display import HTML
from base64 import b64encode
import gymnasium as gym
from gym.wrappers import RecordVideo
from stable_baselines3.ppo.policies import MlpPolicy
from stable_baselines3 import DQN
from stable_baselines3 import PPO, SAC, TD3
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
import os
import renderlab as rl
from stable_baselines3.common.vec_env import DummyVecEnv
import time



In [ ]:
display = Display(visible=False, size=(1400, 900))
display.start()

def render_video(video_path):
    video = open(video_path, 'rb').read()
    video_encoded = b64encode(video).decode()
    return HTML(f'<video width=400 controls><source src="data:video/mp4;base64,{video_encoded}" type="video/mp4"></video>')

env = gym.make("CarRacing-v2", render_mode="rgb_array")
env = RecordVideo(env, './video', episode_trigger=lambda e: True)

# Run a few episodes to render the environment
episodes = 2
for episode in range(1, episodes + 1):
    state, info = env.reset()
    done = False
    score = 0

    while not done:
        env.render()  # This will now render via the virtual display
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward

env.close()

# Render the video from the first episode
render_video('./video/rl-video-episode-0.mp4')


In [ ]:
env = gym.make("CarRacing-v2")
env = DummyVecEnv([lambda: env])
log_path = ('Training', 'Logs')
model = SAC('CnnPolicy', env, verbose=1)
model.learn(total_timesteps=30000)

PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model_cnnPolicy')
model.save(PPO_path)


In [ ]:
#env = gym.make("CarRacing-v2")
model = PPO.load('/content/PPO_2m_Driving_model_2.7z', env=env)

In [ ]:
# evaluate_policy(model, env, n_eval_episodes=10, render=True)
# Function to render video
def render_video(video_path):
    video = open(video_path, 'rb').read()
    video_encoded = b64encode(video).decode()
    return HTML(f'<video width=400 controls><source src="data:video/mp4;base64,{video_encoded}" type="video/mp4"></video>')

# Start virtual display
display = Display(visible=False, size=(1400, 900))
display.start()


# Record the video of the evaluation
env = gym.make("CarRacing-v2", render_mode="rgb_array")
env = RecordVideo(env, './video', episode_trigger=lambda e: True)

# Evaluate the policy for 10 episodes and render the environment for N seconds (let's say N=10 seconds)
n_eval_episodes = 1
n_seconds = 40

# Run the evaluation with rendering for the first N seconds
obs, info = env.reset()
done = False
start_time = time.time()

for _ in range(n_eval_episodes):
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)

        # Render and stop after N seconds
        env.render()
        if time.time() - start_time > n_seconds:
            break

    # Reset environment for the next episode
    done = False
    obs, info = env.reset()

# Close the environment
env.close()

# Render the video from the first evaluation episode
render_video('./video/rl-video-episode-0.mp4')
